In [3]:
import os
os.environ["OMP_NUM_THREADS"] = "8"  # set the number of CPU threads to use for parallel regions

from pathlib import Path
import numpy as np
import pandas as pd
import time
import heaan_sdk as heaan
import math

# set key_dir_path
key_dir_path = Path('./BinaryDT/keys')

# set parameter
params = heaan.HEParameter.from_preset("FGb")

# init context and load all keys
# if set generate_keys=True, then make key
# if set generate_keys=False, then not make key. just use existing key.
context = heaan.Context(
    params,
    key_dir_path=key_dir_path,
    load_keys="all",
    generate_keys=False,
)

/tmp/ipykernel_2606/4139506428.py:20: UserWarning: The currently available memory size is 4. It must be at least 16GB.
  context = heaan.Context(


The currently used HEaaN library does not support GPU acceleration


In [4]:
num_slot = context.num_slots
num_slot

32768

In [3]:
log_num_slot = context.log_slots
log_num_slot

15

In [12]:
# you can see all methods in Block class
# help(heaan.Block)

### Encrypt & Decrypt
#### Save ctxt file and load

In [8]:
a = [1,2,3,4] # no need to fill 0 to make list length to num_slot
b = [0.1,0.2,0.3,0.4]

## if set encrypted = False, a will not be encrypted. just encoding list to message Block
a = heaan.Block(context,encrypted = False, data = a)
b = heaan.Block(context,encrypted = False, data = b)

## encrypt 

ctxt1 = a.encrypt()
ctxt2 = b.encrypt()

## save ciphertext to ctxt file format
ctxt1.save('/root/tutorial/python/x1.ctxt')
ctxt2.save('/root/tutorial/python/x2.ctxt')

Ciphertext(log(num_slot): 15, device: CPU, level: 12)

In [11]:
## load ctxt file

## make emtpy ctxt
empty_msg= heaan.Block(context,encrypted = False) # data : default is None
load_ctxt1 = empty_msg.encrypt()   
load_ctxt2 = empty_msg  ## after call 'empty_msg.encrypt()', then 'empty_msg' is encrypted as ciphertext

load_ctxt1 = load_ctxt1.load('/root/tutorial/python/x1.ctxt')
load_ctxt2 = load_ctxt2.load('/root/tutorial/python/x2.ctxt')

## Ciphertext print Option1. 
print_ctxt(load_ctxt1,5)
print(' ')
print_ctxt(load_ctxt2,5)
print(' ')

## Ciphertext print Option2. 
x1 = load_ctxt1.decrypt() # then load_ctxt1 is decrypted as message (it is no longer ciphertext)
x2 = load_ctxt2.decrypt()

for i in range(5):
    print('x1: ',x1[i])
    print('x2: ',x2[i])

0 (1.0000000033515675+2.1711986300199003e-09j)
1 (2.000000012352754-4.636489161465599e-09j)
2 (2.9999999742533454+1.4387558553282273e-09j)
3 (3.999999995041752-2.0825264392909296e-08j)
4 (-4.926269784134032e-09-4.547855552527933e-09j)
 
0 (0.10000001542253958-9.081722455021433e-09j)
1 (0.20000000712437488+5.7562660335395475e-09j)
2 (0.300000006809978+8.943145486403779e-09j)
3 (0.39999999744233394+4.0791093476541065e-08j)
4 (1.4968096319438594e-08-1.210753429035947e-09j)
 
x1:  (1.0000000033515675+2.1711986300199003e-09j)
x2:  (0.10000001542253958-9.081722455021433e-09j)
x1:  (2.000000012352754-4.636489161465599e-09j)
x2:  (0.20000000712437488+5.7562660335395475e-09j)
x1:  (2.9999999742533454+1.4387558553282273e-09j)
x2:  (0.300000006809978+8.943145486403779e-09j)
x1:  (3.999999995041752-2.0825264392909296e-08j)
x2:  (0.39999999744233394+4.0791093476541065e-08j)
x1:  (-4.926269784134032e-09-4.547855552527933e-09j)
x2:  (1.4968096319438594e-08-1.210753429035947e-09j)


In [10]:
def check_boot(x):
    if x.level==3:
        x.bootstrap()
    elif x.level<3:
        exit(0)
    return x

def print_ctxt(c,size):
    m = c.decrypt(inplace=False)
    for i in range(size):
        print(i,m[i])
        if (math.isnan(m[i].real)):
            print ("nan detected..stop")
            exit(0)

### Main calculation
#### Ciphertext + number

In [12]:
block = heaan.Block(context,encrypted = False, data = [0]*num_slot) ## 이렇게하면 block은 ciphertext가 아니라 message임

ctxt = block.encrypt()

## add 1 for all slots of 
ctxt = ctxt + 1

# print(ctxt)
ctxt.decrypt()
for i in range(5):
    print(ctxt[i].real)


0.9999999922476959
0.9999999887624403
1.0000000064372236
1.0000000303167669
0.999999995737224


#### inverse

In [13]:
## msg * ctxt
a= [0.1,0.01,0.001,0.0001]

a_block = heaan.Block(context, data = a) 
ctxt1 = a_block.encrypt()

inverse = ctxt1.inverse(greater_than_one = False)
inverse.decrypt()
for i in range(5):
    print(inverse[i].real)

10.000000315718353
99.99990186541982
999.9876863526733
10001.115531396397
262179.98832135496


#### add, sub, mult

In [14]:
a= [1,2,3,4]
b= [1,2,3,4] 

a_block = heaan.Block(context, data = a) ## 이렇게하면 block은 ciphertext가 아니라 message임
ctxt1 = a_block.encrypt()
b_block = heaan.Block(context, data = b) ## 이렇게하면 block은 ciphertext가 아니라 message임
ctxt2 = b_block.encrypt()

## add
add_ctxt = ctxt1 + ctxt2

## sub
sub_ctxt = ctxt1 - ctxt2

## mult
# excute mult operation, then cihertext level 1 down
# init level of ciphertext = 12
# print(ctxt1.level) ## 12
mult_ctxt = ctxt1 * ctxt2
# print(mult_ctxt.level) ## 11

#### rotate
you don't need to rot_idx as block. it can be just int value.

In [15]:
a = [0.1, 0.2, 0.3, 0.4]
a = heaan.Block(context,data=a)
a.encrypt()
print(a)

rot_idx = 1
res = a.__lshift__(rot_idx) # left rotate 'a' ciphertext 'rot_idx' slots 

res.decrypt()
print(res[0])
print(res[1])
print(res[2])
print(res[3])
print(res[4])

res2 = a.__rshift__(rot_idx) # right rotate 'a' ciphertext 'rot_idx' slots 
res2.decrypt()
print(' ')
print(res2[0])
print(res2[1])
print(res2[2])
print(res2[3])
print(res2[4])

Ciphertext(log(num_slot): 15, device: CPU, level: 12)
(0.19999999320903158+3.2849852222637134e-09j)
(0.3000000074784588-8.453653715750744e-09j)
(0.40000004558468816-2.440390060491175e-09j)
(1.836140808337516e-08-2.7250364062917074e-09j)
(-7.84106134746879e-09+3.401902441924129e-08j)
 
(3.250580490950235e-08-2.0899486241772277e-08j)
(0.10000000904632533-6.527107664591949e-10j)
(0.19999999265960336+5.034915853588839e-09j)
(0.3000000071572689-7.872942290715758e-09j)
(0.4000000450399934-2.1893719406148973e-09j)


In [16]:
help(heaan.Block.__neg__)
# res = self.__neg__()

Help on cython_function_or_method in module heaan_sdk.block:

__neg__(self) -> 'Block'



In [17]:
self.decrypt()
print(self[0])
print(self[1])
print(self[2])
print(self[3])
print(self[4])

NameError: name 'self' is not defined

#### negate
change the sign (ex. -2 -> +2)

In [ ]:
li = [2,0,-2]+[0]*(num_slot-3)
li = heaan.Block(context, data=li)
li.encrypt()

res = 0 - li ## 0 - ciphertext is also make same result of negate method
res2 = li.__neg__()
res.decrypt()
print(res[0])
print(res[1])
print(res[2])
print(res[3])
print(res[4])
print('')

res2.decrypt()
print(res2[0])
print(res2[1])
print(res2[2])
print(res2[3])
print(res2[4])

#### sign
- 양수면 1, 음수면 -1 나오게 하는 함수
- log_range : Log of the input range. Integer <= 38. Defaults to 0, which means that the domain of approximation is [-1, 1].

In [ ]:
li = [0.2,0,-0.2]
li = heaan.Block(context, data=li)
res= li.encrypt()

res.sign(inplace = True, log_range=0)

res.decrypt()
print(res[0])
print(res[1])
print(res[2])
print(res[3])
print(res[4])

#### greater than zero
- 음수 면 0, 양수면 1, 0이면 0.5

In [18]:
li = [-0.1,0.2,0.5]
li = heaan.Block(context, data=li)
li= li.encrypt()

res= li.greater_than_zero()

res.decrypt()
print(res[0])
print(res[1])
print(res[2])
print(res[3])
print(res[4])

(3.849505991748714e-10+6.773722927550588e-11j)
(0.9999999992998507-1.1215461761812492e-10j)
(1.000000002811444+8.352405424189487e-10j)
(0.4948779082648287-2.026023887153482e-09j)
(0.49723285326616995+1.5938540162428722e-08j)


In [78]:
context.min_level_for_bootstrap

3

#### rotate_reduce

In [4]:
def left_rotate_reduce(context,data,gs,interval):
    # data = Block
   
    # m0 = heaan.Message(logN-1,0)
    m0 = heaan.Block(context,encrypted = False, data = [0]*context.num_slots)
    res = m0.encrypt()
    
    empty_msg= heaan.Block(context,encrypted = False)
    rot = empty_msg.encrypt(inplace=False)
    
    binary_list = []
    while gs > 1:
        if gs%2 == 1:
            binary_list.append(1)
        else:
            binary_list.append(0)
        gs = gs//2
    binary_list.append(gs)

    i = len(binary_list)-1
    sdind = 0
    while i >= 0:
        if binary_list[i] == 1:
            ind = 0
            s = interval
            tmp = data
            while ind < i:
            
                rot = tmp.__lshift__(s)
                tmp = tmp + rot
                s = s*2
                ind = ind+1
            if sdind > 0:
                tmp = tmp.__lshift__(sdind)
            res = res + tmp
            sdind = sdind + s
        i = i - 1            

    del  rot, tmp
    
    return res

In [5]:
n = 2
d = 3
data = [1]*32768
msg = heaan.Block(context, data = data,encrypted=False)
tmp = msg.encrypt()

result = left_rotate_reduce(context,tmp,n*d,1)
result.decrypt()
for i in range(10):
    print(result[i])

(5.999999944596432+1.0537988438416347e-08j)
(5.999999894948648-2.4197436735102753e-08j)
(5.999999904658025+1.81116828749206e-08j)
(5.999999911223057+1.5890127989352557e-08j)
(5.99999997197177-3.4715969437379393e-09j)
(6.000000008247307-1.6302118821967013e-08j)
(6.000000013648723+1.5780694522419233e-08j)
(6.0000000657278605-2.383036130835191e-09j)
(6.000000040475667-3.333911646479907e-08j)
(6.000000042551077-3.047243636354767e-08j)
